In [1]:
# Standard python imports
%matplotlib inline

from os.path import join
import glob
import inspect
from importlib import reload
import warnings

from matplotlib.colors import LogNorm

import numpy as np
import pylab as plt
import pandas as pd

import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

# data structures from astropy
import astropy.io.fits as fits
from astropy.table import Table, vstack

# astronomical image fitting software Imfit, wrapped in Python
# https://github.com/perwin/pyimfit
# https://pyimfit.readthedocs.io/en/latest/
import pyimfit

# MCMC sampling package
import emcee

# necessary utilities from scipy, astropy, and photutils
from scipy.ndimage import maximum_filter
from scipy.stats import multivariate_normal
from scipy.special import gammainc, gamma
from scipy.optimize import minimize_scalar
from astropy.stats import SigmaClip, gaussian_fwhm_to_sigma
from astropy.convolution import Gaussian2DKernel
from photutils.background import Background2D, StdBackgroundRMS
from photutils.segmentation import detect_sources, SegmentationImage, deblend_sources

# plots
import matplotlib.pyplot as plt
from matplotlib import rc
from mpl_toolkits.axes_grid1 import make_axes_locatable

import sys
sys.path.append('/Users/aidan/Desktop/sl_project/galaxy-galaxy-sel/')

from imfit_mcmc import SersicMCMC

In [2]:

# the following commands make plots look better
def plot_prettier(dpi=200, fontsize=10): 
    plt.rcParams['figure.dpi']= dpi
    plt.rc("savefig", dpi=dpi)
    plt.rc('font', size=fontsize)
    plt.rc('xtick', direction='in')
    plt.rc('ytick', direction='in')
    plt.rc('xtick.major', pad=5) 
    plt.rc('xtick.minor', pad=5)
    plt.rc('ytick.major', pad=5) 
    plt.rc('ytick.minor', pad=5)
    plt.rc('lines', dotted_pattern = [2., 2.])
    # if you don't have LaTeX installed on your laptop and this statement 
    # generates error, comment it out
    rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
    rc('text', usetex=True)
    
plot_prettier()

In [3]:
def read_fits_table(filename):
    '''
    reads and returns data in a table from a FITS file
    '''
    hdu = fits.open(filename)
    data = Table(hdu[1].data)   # second index of the hdu corresponds to the data in my astropy table files
    hdu.close()
    
    return data

def read_fits_image(filename):
    '''
    reads and returns an image from a FITS file
    '''
    hdu = fits.open(filename)
    data = hdu[0].data
    header = hdu[0].header 
    hdu.close()
    
    return data

In [70]:
# get table of redmagic data
des_data = read_fits_table('/Users/aidan/Desktop/sl_project/results/redmagic_4kqf_test.fits')

sl_data98 = read_fits_table('/Users/aidan/Desktop/sl_project/results/fitting_98lenses_test.fits')
sl_se_data98 = pd.read_csv('/Users/aidan/Desktop/sl_project/catalogs/jacobs2019_aidan_cut_dr2.csv')

sl_data505 = pd.read_csv('/Users/aidan/Desktop/sl_project/catalogs/jacobs2019_dr2.csv')

In [71]:
# DES magnitudes
des_r = des_data['mag_r']
des_g = des_data['mag_g']

sl98_r = sl_data98['mag_r']
sl98_g = sl_data98['mag_g']

sl505_r = sl_data505['mag_auto_r']
sl505_g = sl_data505['mag_auto_g']

# DES colors
des_gr = des_g - des_r
sl98_gr = sl98_g - sl98_r
sl505_gr = sl505_g - sl505_r

In [72]:

des_filt = (np.isfinite(des_r) & np.isfinite(des_g) & np.isfinite(des_gr))
des_r_filt = des_r[des_filt]
des_g_filt = des_g[des_filt]
des_gr_filt = des_gr[des_filt]

des_ellip_filt = des_data['er'][des_filt]

# DES Sersic photometry parameters

plate_scale = 0.263                                 # arcsec per pixel

des_nr = des_data['nr'][des_filt]                   # Sersic index
des_rr = des_data['Rr'][des_filt] * plate_scale     # half-light radius (in arcsec)
des_ng = des_data['ng'][des_filt]
des_rg = des_data['Rg'][des_filt] * plate_scale

sl_nr = sl_data98['nr']                               # Sersic index
sl_rr = sl_data98['Rr'] * plate_scale                 # half-light radius (in arcsec)
sl_ng = sl_data98['ng']
sl_rg = sl_data98['Rg'] * plate_scale

sl_ellip = sl_data98['er']

# residual flux values
# this still needs to be scaled with uncertainty, which I need to implement a calculation for

des_resflux_r = des_data['obs_flux_35pm3_r'] - des_data['mod_flux_35pm3_r']
des_resflux_g = des_data['obs_flux_35pm3_g'] - des_data['mod_flux_35pm3_g']

sl_resflux_r = (sl_data98['obs_flux_35pm3_r'] - sl_data98['mod_flux_35pm3_r']) #/ sl_data98['mod_flux_35pm3_err_r']
sl_resflux_g = (sl_data98['obs_flux_35pm3_g'] - sl_data98['mod_flux_35pm3_g']) #/ sl_data98['mod_flux_35pm3_err_g']


In [73]:
from scipy.stats import ks_2samp

ks_2samp(des_nr, sl_nr)

Ks_2sampResult(statistic=0.2645184389545292, pvalue=2.1437561149584283e-06)

In [74]:
ks_2samp(des_rr, sl_rr)

Ks_2sampResult(statistic=0.5303616183315432, pvalue=0.0)

In [75]:
ks_2samp(des_ng, sl_ng)

Ks_2sampResult(statistic=0.4950948800572861, pvalue=0.0)

In [76]:
ks_2samp(des_rg, sl_rg)

Ks_2sampResult(statistic=0.2577873254564984, pvalue=4.302881795426039e-06)

In [77]:
ks_2samp(des_r, sl98_r)

Ks_2sampResult(statistic=0.2209591836734694, pvalue=0.00013924723820057938)

In [78]:
ks_2samp(des_g, sl98_g)

Ks_2sampResult(statistic=0.25407142857142856, pvalue=6.253968968361612e-06)

In [79]:
ks_2samp(des_gr, sl98_gr)

Ks_2sampResult(statistic=0.1678469387755102, pvalue=0.007965830623416048)

In [80]:
ks_2samp(des_ellip_filt, sl_ellip)

Ks_2sampResult(statistic=0.13243823845327604, pvalue=0.06347380250682833)